In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from astropy.constants import G, M_sun
from astropy.modeling import models, fitting
from astropy.timeseries import LombScargle
from uncertainties import ufloat
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [2]:
def radial_velocity_model(t, K, e, omega_deg, P, T0, gamma):
    omega = np.radians(omega_deg)
    n = 2 * np.pi / P  # Mean motion
    M = n * (t - T0)  # Mean anomaly

    # Solve Kepler's Equation: M = E - e * sin(E)
    E = M.copy()
    for _ in range(100):  # Iterate to solve for E
        E_prev = E
        E = M + e * np.sin(E)
        if np.max(np.abs(E - E_prev)) < 1e-10:
            break

    # Compute true anomaly
    nu = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2),
                        np.sqrt(1 - e) * np.cos(E / 2))

    # Radial velocity
    V = gamma + K * (np.cos(omega + nu) + e * np.cos(omega))
    return V

In [4]:
# Define the fitting function
def fit_func(t, K1, e, omega_deg, P, T0, gamma):
    return radial_velocity_model(t, K1, e, omega_deg, P, T0, gamma)

In [3]:
# Initial parameter guesses
initial_params = {
    'K1': 200,      # Semi-amplitude for star 1 (km/s)
    'e': 0.0,       # Eccentricity
    'omega_deg': 0, # Argument of periapsis (degrees)
    'P': 2.7,       # Orbital period (days)
    'T0': data['Time'].min(),  # Time of periastron passage
    'gamma': 0      # Systemic velocity (km/s)
}

NameError: name 'data' is not defined

In [ ]:
# Fit the model
popt, pcov = curve_fit(
    fit_func,
    t,
    RV1,
    sigma=RV1_err,
    absolute_sigma=True,
    p0=list(initial_params.values()),
    bounds=(
        [0, 0, 0, 0, t.min(), -np.inf],
        [np.inf, 1, 360, np.inf, t.max(), np.inf]
    )
)